In [2]:
import os
from pathlib import Path
import pandas as pd
from pandas_datareader import data as pdr
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import yfinance as yf


# plotting configuration
plt.rcParams.update({
    'figure.figsize': (11,6),
    'figure.dpi': 120,
    'axes.grid': True,
    'grid.alpha': 0.25,
})


# make output directories
OUT_DIR = Path("outputs")
FIG_DIR = OUT_DIR / "figs"
OUT_DIR.mkdir(exist_ok=True)
FIG_DIR.mkdir(exist_ok=True)

C:\Users\user\Documents\Code\AI Genesis Htn\NC_env\Lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


In [3]:
# TICKER mapping: 
TICKERS = {
    'SPY': 'SPY',
    'QQQ': 'QQQ',
    'VXUS': 'VXUS',
    'BND': 'BND',
    'IEF': 'IEF',
    'TIP': 'TIP', # Yahoo ticker for TIPS ETF is TIP
    'SHY': 'SHY',
    'GLD': 'GLD',
    # Crypto
    'BTC': 'BTC-USD',
    'ETH': 'ETH-USD',
    'SOL': 'SOL-USD'
}


START_DATE = '1995-01-01' # covers optional 1997 event
END_DATE = dt.date.today().isoformat()


# portfolio sleeve targets
TOTAL_TARGET = {'equity': 0.60, 'bond': 0.30, 'cash': 0.10, 'gold': 0.0}
EQUITY_BREAKDOWN = {'SPY': 0.7, 'QQQ': 0.2, 'VXUS': 0.1}
BOND_BREAKDOWN = {'BND': 0.6667, 'IEF': 0.16665, 'TIP': 0.16665}
CASH_ASSET = 'SHY'
GOLD_ASSET = 'GLD'


# whether to include crypto variant
INCLUDE_CRYPTO_VARIANT = False


# Event windows 
EVENTS = {
    "Dotcom": ("2000-03-01", "2002-10-31"),
    "GFC": ("2007-10-01", "2009-03-31"),
    "COVID": ("2020-02-01", "2020-04-30"),
    "2022_rates": ("2022-01-01", "2022-10-31"),
    
    "Asian_1997": ("1997-07-01", "1998-12-31"),
    "Debt_Ceiling_2011": ("2011-08-01", "2011-12-31"),
}


# --------------------
# small helper flags
VERBOSE = True

## Helper functions: download data, simulate monthly rebalancing, compute metrics, plotting helpers.

In [4]:
def download_prices(tickers_map, start, end):
    """
    Download Close prices (price-only) from Yahoo for each symbol in tickers_map.
    Returns a DataFrame with friendly names as columns and a DateTime index.
    
    
    This implementation downloads all tickers in a single yf.download call (faster, more robust),
    handles the single-ticker edge case, and renames columns to the friendly keys used everywhere
    in this notebook (e.g. 'SPY', 'BND', etc.).
    """
    yahoo_symbols = list(tickers_map.values())
    print("Downloading:", ", ".join(yahoo_symbols))
    raw = yf.download(yahoo_symbols, start=start, end=end, progress=False)
    
    
    # raw may be a DataFrame with multi-level columns when multiple fields requested; we asked for 'Close'
    if isinstance(raw, pd.DataFrame) and 'Close' in raw.columns:
        df = raw['Close'].copy()
    else:
        # sometimes yf.download returns a DataFrame of closes directly or a Series for single symbol
        df = raw.copy()
    
    
    # convert Series -> DataFrame if necessary
    if isinstance(df, pd.Series):
        df = df.to_frame()
    
    
    # rename columns using reverse mapping (yahoo ticker -> friendly name)
    revmap = {v: k for k, v in tickers_map.items()}
    # if df columns are yahoo tickers, map them; otherwise keep as-is
    df = df.rename(columns=lambda c: revmap.get(c, c))
    
    
    df.index = pd.to_datetime(df.index)
    df = df.sort_index().ffill().dropna(how='all')
    return df


# Download prices once (single source of truth for the notebook)
prices = download_prices(TICKERS, START_DATE, END_DATE)
print("Downloaded columns:", list(prices.columns))
# quick preview
display(prices.iloc[-5:])

Downloading: SPY, QQQ, VXUS, BND, IEF, TIP, SHY, GLD, BTC-USD, ETH-USD, SOL-USD


C:\Users\user\AppData\Local\Temp\ipykernel_27600\4107393473.py:13: FutureWarning: YF.download() has changed argument auto_adjust default to True
  raw = yf.download(yahoo_symbols, start=start, end=end, progress=False)

11 Failed downloads:
['BTC-USD', 'ETH-USD', 'VXUS', 'QQQ', 'GLD', 'IEF']: Timeout('Failed to perform, curl: (28) Connection timed out after 30001 milliseconds. See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')
['TIP', 'SPY', 'SHY', 'SOL-USD', 'BND']: Timeout('Failed to perform, curl: (28) Connection timed out after 30002 milliseconds. See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')


Downloaded columns: ['BND', 'BTC', 'ETH', 'GLD', 'IEF', 'QQQ', 'SHY', 'SOL', 'SPY', 'TIP', 'VXUS']


Ticker,BND,BTC,ETH,GLD,IEF,QQQ,SHY,SOL,SPY,TIP,VXUS
Date,,,,,,,,,,,


## Compute daily returns from Price series. We use price-only returns per the team's instruction.


In [28]:
returns = prices.pct_change()
returns = returns.dropna(how='all')


# preview
returns.iloc[-5:].round(6)

Ticker,BND,BTC,GLD,IEF,QQQ,SHY,SPY,TIP,VXUS
Date,,,,,,,,,
2025-08-27,0.000949,-0.005193,0.002019,0.001878,0.001537,0.000724,0.002279,0.002253,-0.002933
2025-08-28,0.001219,0.011893,0.007419,0.002291,0.006260,-0.000362,0.003541,0.000629,0.004062
2025-08-29,-0.001218,-0.036732,0.009650,-0.000831,-0.011576,0.000844,-0.005964,-0.001078,-0.004464
2025-08-30,0.000000,0.003664,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2025-08-31,0.000000,-0.005251,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
